In [64]:
import pandas as pd
import numpy as np
import os
import itertools

In [65]:
os.chdir('c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL')
os.getcwd()

'c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL'

In [66]:
scores = pd.read_csv('backend/data/games/scores.csv')
scores['date'] = pd.to_datetime(scores['date'])
scores['first_score_time'] = scores['first_score_time'].apply(lambda x: int(x.split(':')[0]) + round(float(x.split(':')[1]) / 60, 2))
scores['first_score_time'] = scores['first_score_time'] + np.where(scores['first_score_quarter'] == 1, 0, 15)
scores['first_score'] = scores['team'].apply(lambda x: x.split(' ')[-1]) == scores['first_score_team']
scores = scores[['date', 'team', 'opponent', 'week', 'season', 'first_score_team', 'first_score_quarter', 'first_score_time', 'first_score']]
scores.sample(5)


,date,team,opponent,week,season,first_score_team,first_score_quarter,first_score_time,first_score
2098,2013-10-20,Denver Broncos,Indianapolis Colts,7,2013,Broncos,1,8.27,True
552,2011-11-20,Arizona Cardinals,San Francisco 49ers,11,2011,49ers,1,3.17,False
3660,2017-01-14,Seattle Seahawks,Atlanta Falcons,19,2016,Seahawks,1,6.43,True
90,2010-12-16,San Francisco 49ers,San Diego Chargers,15,2010,Chargers,1,12.88,False
3688,2016-10-02,Seattle Seahawks,New York Jets,4,2016,Jets,1,6.45,False


In [67]:
df = scores[scores['season'] == 2021].groupby(['team', 'first_score']).agg({'first_score_time': 'mean', 'first_score': 'count'})
df = df.join(scores[scores['season'] == 2021].groupby(['team'])['week'].count(), how='inner')
df['accuracy'] = round(df['first_score'] / df['week'], 2)
df.head(10)

first_score_time  first_score  week  accuracy
team              first_score                                               
Arizona Cardinals False                7.988571            7    18      0.39
                  True                 8.483636           11    18      0.61
Atlanta Falcons   False                8.247778            9    17      0.53
                  True                 7.821250            8    17      0.47
Baltimore Ravens  False               12.314286            7    17      0.41
                  True                 9.815000           10    17      0.59
Buffalo Bills     False                8.496667            6    19      0.32
                  True                10.191538           13    19      0.68
Carolina Panthers False               14.244000            5    17      0.29
                  True                11.645833           12    17      0.71

In [68]:
teams = ('Buffalo Bills', 'Los Angeles Rams')
first_score = (True, False)
bills_first = df.loc[(teams[0], first_score[0]), :]['accuracy'] * df.loc[(teams[1], first_score[1]), :]['accuracy']
rams_first = df.loc[(teams[0], first_score[1]), :]['accuracy'] * df.loc[(teams[1], first_score[0]), :]['accuracy']

In [69]:
round(bills_first / (bills_first + rams_first) * 100, 1)

51.1

In [70]:
round(rams_first / (bills_first + rams_first) * 100, 1)

48.9